In [1]:
!pip install langchain groq
!pip install langchain-groq groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 656.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [28]:
from langchain_groq import ChatGroq
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain import PromptTemplate, LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os


# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the Groq LLM
llm = ChatGroq(model_name="mixtral-8x7b-32768")


##Question 1: Chain-of-Thought Prompting

Implement a Chain-of-Thought prompting system to solve word problems. Your system should break down the problem-solving process into steps and show its reasoning.

Task:

● Create a function that takes a word problem as input.

● Use the OpenAI API or any other LLM API to generate a step-by-step solution.

● The output should show each step of the reasoning process.

● Test your function with at least two word problems of varying complexity.

Example word problem: "A baker has 150 eggs. If each cake requires 3 eggs and each cookie
requires 2 eggs, how many cakes and cookies can the baker make if they want to use all the
eggs and make an equal number of each?"

In [37]:
def chain_of_thought_solver(word_problem):
    # Setup SystemMessage and HumanMessage
    system_message = SystemMessage(content="You are a helpful assistant. Break down the problem into steps and show your reasoning.")

    # Combine the system message with the human message containing the word problem
    human_message = HumanMessage(content=f"Question: {word_problem}\nAnswer:")

    # Combine the messages into a prompt template
    prompt_template = ChatPromptTemplate.from_messages([system_message, human_message])

    # Create an LLMChain with the prompt template
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Run the chain to generate the response
    response = chain.run({})

    return response


# Test the function with two word problems

problem1 = "A baker has 150 eggs. If each cake requires 3 eggs and each cookie requires 2 eggs, how many cakes and cookies can the baker make if they want to use all the eggs and make an equal number of each?"

problem2 = "If a car travels at a speed of 60 km/h for 2 hours, how far does it travel, and if the car then slows down to 40 km/h for 1 hour, what is the total distance traveled?"

# Get the solutions
solution = chain_of_thought_solver(problem2)

print(f"Problem 2 Solution:\n{solution}\n")



Problem 2 Solution:
Sure, let's break this down:

1. First, we need to calculate how far the car travels at 60 km/h for 2 hours. Speed multiplied by time gives distance, so we have: 60 km/h * 2 h = 120 km. This means the car travels 120 kilometers in the first part of the trip.

2. Next, we need to calculate how far the car travels at 40 km/h for 1 hour. Again, using the formula distance = speed * time, we have: 40 km/h * 1 h = 40 km. This means the car travels an additional 40 kilometers in the second part of the trip.

3. Finally, to find the total distance traveled, we add the distances from both parts of the trip: 120 km (from the first part) + 40 km (from the second part) = 160 km.

So, the car travels a total of 160 kilometers.



##Question 2: Prompt-chaining

Develop a prompt-chaining system that generates a short story based on user input. Thesystem should use multiple prompts in sequence, with each prompt building on the output of theprevious one.

Task:

● Create a series of at least 3 prompts that will be chained together.

● The first prompt should ask for a genre and main character.

● Subsequent prompts should build on previous outputs to develop the plot and
conclusion.

● Implement the chaining logic to pass information between prompts.

● Allow the user to input the initial genre and main character.

● Output the final story generated through the chain.


In [31]:
first_prompt = PromptTemplate(
    input_variables=["genre", "main_character"],
    template="Write the beginning of a {genre} story where the main character is {main_character}."
)

second_prompt = PromptTemplate(
    input_variables=["story_beginning"],
    template="Based on this beginning, continue the story and develop the main conflict: {story_beginning}"
)

third_prompt = PromptTemplate(
    input_variables=["story_plot"],
    template="Given the plot, provide a satisfying conclusion to the story: {story_plot}"
)

# Create the LLMChains
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_three = LLMChain(llm=llm, prompt=third_prompt)

def generate_story(genre, main_character):
    # Generate the beginning of the story
    story_beginning = chain_one.run({"genre": genre, "main_character": main_character})
    print(f"Story Beginning:\n{story_beginning}\n")

    # Generate the plot based on the beginning
    story_plot = chain_two.run({"story_beginning": story_beginning})
    print(f"Story Plot:\n{story_plot}\n")

    # Generate the conclusion based on the plot
    final_story = chain_three.run({"story_plot": story_plot})
    return final_story

# User input for the genre and main character
genre = input("Enter the genre of the story: ")
main_character = input("Enter the main character's name: ")

# Generate and print the final story
final_story = generate_story(genre, main_character)
print(f"Generated Story:\n{final_story}")


Enter the genre of the story: comedy
Enter the main character's name: vijay
Story Beginning:
Vijay was having a perfectly ordinary day until he received a mysterious letter in the mail. "Congratulations, you have been selected to participate in the Great Indian Talent Show!" it read. Vijay was puzzled - he wasn't a singer, dancer, or actor. In fact, he was a pretty average guy who worked as an accountant at a small firm in Mumbai.

Despite his confusion, Vijay couldn't help but feel a spark of excitement. He had always dreamed of doing something out of the ordinary, something that would make people laugh. And so, he decided to take a chance and audition for the talent show.

On the day of the audition, Vijay was a nervous wreck. He had spent hours rehearsing his stand-up comedy routine, but he couldn't shake the feeling that he was going to bomb.

As he took the stage, the audience looked at him with a mix of curiosity and skepticism. But Vijay was determined to win them over. He launc

##Question 3: Tree-of-Thought Prompting

Implement a Tree-of-Thought prompting system to solve a complex problem. The system should
explore multiple paths of reasoning and select the most promising one.
Task:

● Create a function that takes a complex problem as input.

● Generate at least 3 different initial approaches to solving the problem.

● For each approach, develop 2-3 possible next steps.

● Implement a method to evaluate the promise of each path.

● Select the most promising path and generate a final solution.

● Test your system with the following problem:

Problem: "Design a sustainable urban transportation system for a city of 1 million people.
Consider factors such as environmental impact, cost-effectiveness, and accessibility for all
residents."

In [12]:
def tree_of_thought(problem):
    # Setup SystemMessage
    system_message = SystemMessage(content="You are a strategic assistant. Generate multiple paths to solve the problem, evaluate them, and select the best one.")

    # Setup HumanMessage with the complex problem
    human_message = HumanMessage(content=f"Problem: {problem}\nGenerate at least 3 different initial approaches, develop 2-3 possible next steps for each approach, evaluate the paths, and select the most promising one.")

    # Combine the messages into a prompt template
    prompt_template = ChatPromptTemplate.from_messages([system_message, human_message])

    # Create an LLMChain with the prompt template
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Run the chain to generate the response
    response = chain.run({})
    return response

# Test the system with the provided problem
problem = "Design a sustainable urban transportation system for a city of 1 million people. Consider factors such as environmental impact, cost-effectiveness, and accessibility for all residents."
solution = tree_of_thought(problem)
print(f"Generated Solution:\n{solution}")

Generated Solution:
Approach 1: Expanding public transportation options
- Next step 1: Invest in a comprehensive network of buses, trams, and trains that cover the entire city.
- Next step 2: Implement a bike-sharing program with stations located near transit stops.
- Next step 3: Encourage ride-sharing and carpooling through incentives and designated lanes.

Approach 2: Encouraging electric vehicles (EVs)
- Next step 1: Install charging infrastructure throughout the city.
- Next step 2: Offer tax incentives for purchasing EVs.
- Next step 3: Partner with local businesses to provide EVs as part of their company fleet.

Approach 3: Promoting active transportation
- Next step 1: Build a network of safe and accessible sidewalks and bike lanes.
- Next step 2: Launch a public awareness campaign about the health benefits of walking and cycling.
- Next step 3: Implement policies that prioritize active transportation, such as reducing speed limits and increasing fines for distracted driving.

